<img src="img/python-logo-no-text.svg"
     style="display:block;margin:auto;width:10%"/>
<br>
<div style="text-align:center; font-size:200%;"><b>Externe Programme</b></div>
<br/>
<div style="text-align:center;">Dr. Matthias Hölzl</div>

## Sub-Prozesse

*Hinweis:* Zur Ausführung dieses Notebooks müssen muss das `ext_sample_app`
Package (in `Examples/ExternalSampleApplication`) installiert sein.

`subprocess.run` ist die bevorzugte Methode um externe Applikationen zu starten.

In [ ]:
from subprocess import TimeoutExpired, run

In [ ]:
# This may not work if `python` is not in your path...
run(["python", "--version"])

Mit `shutil.which()` kann man den vollständigen Pfad eines Programms herausfinden.

In [ ]:
import shutil

shutil.which("python")

In [ ]:
cp = run([shutil.which("python"), "--version"])

In [ ]:
def print_completed_process(cp):
    print("return code:", cp.returncode)
    print("captured stdout:", repr(cp.stdout))
    print("captured stderr:", repr(cp.stderr))

In [ ]:
print_completed_process(cp)

In [ ]:
cp = run([shutil.which("python"), "--version"], capture_output=True, text=True)

In [ ]:
print_completed_process(cp)

Mit `sys.executable` kann man den Pfad des gerade aktiven Python Interpreters herausfinden. Das ist die bevorzugte Methode um einen Python Prozess zu starten.

In [ ]:
import sys

cp = run([sys.executable, "--version"], capture_output=True, text=True)

In [ ]:
print_completed_process(cp)

In [ ]:
cp = run([sys.executable, "-m", "ext_sample_app"], capture_output=True, text=True)

In [ ]:
print_completed_process(cp)

In [ ]:
cp = run(
    [sys.executable, "-m", "ext_sample_app", "--help"], capture_output=True, text=True
)

In [ ]:
print_completed_process(cp)

In [ ]:
cp = run(
    [sys.executable, "-m", "ext_sample_app", "say-hi"], capture_output=True, text=True
)

In [ ]:
print_completed_process(cp)

In [ ]:
cp = run(
    [sys.executable, "-m", "ext_sample_app", "error"], capture_output=True, text=True
)

In [ ]:
print_completed_process(cp)

In [ ]:
# THIS DOES NOT WORK!
# cp = run(
#     [sys.executable, "-m", "ext_sample_app", "interact"], capture_output=True, text=True
# )

## Popen: Nebenläufige Ausführung von Programmen

Wenn man nicht warten kann, bis das gestartete Programm beendet wird muss man die `subprocess.Popen` Klasse verwenden:

In [ ]:
from subprocess import Popen, PIPE
import sys

In [ ]:
proc = Popen(
    [sys.executable, "-m", "ext_sample_app", "interact"],
    stdin=PIPE,
    stderr=PIPE,
    stdout=PIPE,
    encoding="utf-8",
    universal_newlines=True,
    bufsize=0,
)

In [ ]:
type(proc)

`proc.communicate()` sendet eine Nachricht and `proc`, schließt die Ein- und Ausgabeströme und beendet den Prozess.

In [ ]:
proc.communicate("work")

Mit `proc.poll()` kann man feststellen, ob der Prozess schon beendet wurde und was der Rückgabewert war. Falls das Ergebniss `None` ist, ist der Prozess noch aktiv. `proc.wait()` wartet eine bestimmte Zeit und gibt den Rückgabewert des Prozesses zurück. Falls der Prozess nicht in der vorgegebenen Zeit beendet wurde, wird eine `TimeoutExpired` Exception ausgelöst.

In [ ]:
proc.poll()

In [ ]:
def run_and_communicate(command):
    proc = Popen(
        [sys.executable, "-m", "ext_sample_app", "interact"],
        stdin=PIPE,
        stderr=PIPE,
        stdout=PIPE,
        encoding="utf-8",
        universal_newlines=True,
        bufsize=0,
    )
    result = proc.communicate(command)
    try:
        wait_result = proc.wait(5)
    except TimeoutExpired:
        print("Process did not terminate!")
        proc.terminate()
        wait_result = proc.wait(5)
    return result, wait_result

In [ ]:
run_and_communicate("work")

In [ ]:
run_and_communicate("exit")

In [ ]:
run_and_communicate("error")

## Kommunikation mit Sockets

Das folgende Beispiel zeigt, wie man einen Prozess starten und dann über Sockets mit ihm kommunizieren kann.

In [ ]:
from subprocess import Popen, PIPE
import sys

HOST = "localhost"
PORT = 12345

In [ ]:
from socket import socket, AF_INET, SOCK_STREAM
import sys


def send_message(msg: str):
    with socket(AF_INET, SOCK_STREAM) as sock:
        sock.connect((HOST, PORT))
        sock.sendall(bytes(msg + "\n", "utf-8"))
        return str(sock.recv(1024), "utf-8")

In [ ]:
proc = Popen(
    [
        sys.executable,
        "-m",
        "ext_sample_app",
        "serve",
        "--host",
        HOST,
        "--port",
        str(PORT),
    ],
    stdin=PIPE,
    stderr=PIPE,
    stdout=PIPE,
    encoding="utf-8",
    universal_newlines=True,
    bufsize=0,
)

In [ ]:
proc.poll()

In [ ]:
send_message("Hello, world!")

In [ ]:
send_message("Are you running?")

In [ ]:
proc.poll()

In [ ]:
proc.terminate()

In [ ]:
proc.poll()

In [ ]:
try:
    send_message("Are you still running?")
except ConnectionRefusedError as err:
    print("Could not connect to server.")
    print(err)

In [ ]:
proc.poll()